In [1]:
import math
import torch
from torch import nn
from pathlib import Path
import pandas as pd

import ipywidgets as widgets
from ipywidgets import GridspecLayout,Dropdown,BoundedFloatText,Button,Layout

MODEL_PATH = Path.cwd()/'Models'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
DEBUG=False

In [2]:
#import this or somehow avoid explicitely writing it here
class NN(nn.Module):
    def __init__(self, layers=[1000,500,250], ps=0.35, in_features=20, y_range=(20, 90)):
        super(NN, self).__init__()
        self.y_range = y_range
        self.layers = layers
        self.ps = ps
        layers = [in_features] + layers
        layers = list(zip(layers, layers[1:]))
        
        l = []
        for layer in layers:
            l.append(nn.Linear(*layer))
            #TODO: play with negative slope koef. of LeakyReLU
            l.append(nn.LeakyReLU())
            l.append(nn.Dropout(ps))
        l.append(nn.Linear(layers[-1][1], 1))

        self.arch = nn.Sequential(*l)
        
    def forward(self, x):
        x = self.arch(x)
        x = (self.y_range[1]-self.y_range[0])*torch.sigmoid(x)+self.y_range[0]
        return x

In [3]:
def example_change(change):
    def change_values(age):
        sex_box.index = None
        sex_box.value = 'Male' if samples[age][0] == 1 else 'Female'
        for i in range(len(samples[age])):
            grid[i+2, 2].value = samples[age][i+1]
        
    if change['type'] == 'change' and change['name'] == 'value':
        if change['new'] == '-':
            sex_box.index = None
            sex_box.value = '-'
            for i in range(len(list(samples.values())[0])):
                grid[i+2, 2].value = 0
        elif float(change['new'].split()[1]) in list(samples.keys()):
            change_values(int(change['new'].split()[1]))

In [4]:
def create_button(description, button_style, height='auto'):
    button = Button(description=description,
                  tooltip=description,
                  button_style=button_style,
                  disabled=True,
                  layout=Layout(height=height, width='auto'))
    button.style.font_weight = 'bold'
    return button

In [5]:
def submit_clicked(b):
    def predict(inputs):
        path = Path.cwd()/'Models'/'adm_state_dict'
        lm = NN()
        lm.load_state_dict(torch.load(path))
        lm.eval()
        inputs = torch.Tensor(inputs)
        return(lm(inputs)-27)

    age_button.button_style='primary'
    markers = [grid[i+2, 2].value for i in range(len(biomarkers))]
    if sex_box.value=='-':
        age_button.description='Please enter your Sex'
        age_button.style.button_color = 'orange'
    elif 0 in markers:
        age_button.description='Please enter missing biomarkers'
        age_button.style.button_color = 'orange'
    else:
        markers.insert(0, int(sex_box.value=='Male'))
        age_button.style.button_color = '#2471a3'
        if DEBUG: age_button.tooltip = str(markers)
        age_button.description='Your biological age is {} '.format(
            round(predict(markers).data.item()))

In [6]:
warn_text='''Age predictor needs all blood marker values and your Sex to make
a prediction. If you are missing values and filling them in, this might have
a dramatic impact on the results. However your entered Height, Weight, and
Smoking Status are not required for estimating your biological age and only
used for record keeping.'''
title_text = ['Deep Biomarkers Of Human Aging: How Old By a Basic Blood Test',
         '''This is a deep-learned predictor of your age made with a DNN
         trained on tens of thousands anonymized human blood tests. <br>
         Enter yourdata below and the model will guess your age.''']
asterics_text = [
    '* This should be in your clinical biochemistry blood test results.',
    '''** Required parameter for minimal prediction accuracy of 70% within
    10 year frame.''',
    '''*** We can not show you reference values before knowing your age
    apriori, so this is only a reference metric.''']

In [7]:
biomarkers=pd.read_csv('Data/non_NHANES/markers_units.csv').values.tolist()

In [8]:
samples = [list(pd.read_csv('Data/non_NHANES/samples_from_aging.csv').iloc[i]) 
           for i in range(len(pd.read_csv(
               'Data/non_NHANES/samples_from_aging.csv')))]
samples = {sample[-1]:sample[:-1] for sample in samples}

In [9]:
header_grid = widgets.GridspecLayout(4, 5, width='auto')

title_html = widgets.HTML(
    value='''<center><h1 style="font-family:verdana"><font color=
    "paleturquoise">{}</h1><hr><h2 style="font-family:verdana">
    {}</h2></center>'''.format(*title_text))

height_box = widgets.Text(
    placeholder='Enter your height (Optional)',
    description='Height:')

weight_box = widgets.Text(
    placeholder='Enter your weight (Optional)',
    description='Weight:')

load_html = widgets.HTML(value = '<center><h3>Load an Example</h3></center>',
                          layout=Layout(top='45px', left='5px'))

sex_box = widgets.Dropdown(options=['-','Male','Female'], description='Sex:')

smoke_box = widgets.Dropdown(options=['-','No','Yes'], description='Smoke:')

load_box = widgets.Dropdown(
    value='-',
    options=['-']+['Male '+str(int(x))+' y/o' for x in list(samples.keys())],
    description=' ',
    layout=Layout(bottom='45px', right='5px'))
load_box.observe(example_change)

header_grid[0:1,1:-1] = title_html 
header_grid[2,1] = height_box
header_grid[2,2] = weight_box
header_grid[2,3] = load_html 
header_grid[3,1] = sex_box
header_grid[3,2] = smoke_box 
header_grid[3,3] = load_box

header_grid

GridspecLayout(children=(HTML(value='<center><h1 style="font-family:verdana"><font color=\n    "paleturquoise"…

In [10]:
grid=widgets.GridspecLayout(len(biomarkers)+3,5,height='950px',width='auto')
HEIGHT = '28px'

grid[0,1:-1] = widgets.HTML(
    value='<center><h4><font color="orange">{}</h4></center>'.format(
        warn_text))

grid[1,1] = create_button('Blood Marker*', 'primary', '50px')
grid[1,2] = create_button('Your Value', 'primary', '50px')
grid[1,3] = create_button('Units and Sample Metric***', 'primary', '50px')

for i, marker in enumerate(biomarkers): 
        grid[i+2,1] = create_button(marker[0], 'info', HEIGHT)
        grid[i+2,3] = create_button(marker[1], 'info', HEIGHT)
        grid[i+2,2] = BoundedFloatText(min=0,
                                        max=1000,
                                        layout=Layout(height='auto',
                                                      width='auto'),
                                        value=None,
                                        step=0.01,
                                        disabled=False)
        
grid[-1,1:] = widgets.HTML(
    value = '''<p><font color="orange">{}</p><p><fontcolor=
    "orange">{}</p><p><font color="orange">{}
    </h4>'''.format(*asterics_text))

grid

GridspecLayout(children=(HTML(value='<center><h4><font color="orange">Age predictor needs all blood marker val…

In [11]:
footer_grid = widgets.GridspecLayout(3,5)

submit_button = Button(description='Submit',
                tooltip='Predict your age with entered biomarkers',
                button_style='primary',
                layout=Layout(height='50px', width='auto', bottom='0px'))
age_button = Button(layout=Layout(height='50px', width='auto', top='15px'))
age_button.style.button_color = '#111111'
age_button.style.font_weight = 'bold'
submit_button.style.font_weight = 'bold'
submit_button.on_click(submit_clicked)
footer_grid[0,2] = submit_button
footer_grid[1,2] = age_button

footer_grid

GridspecLayout(children=(Button(button_style='primary', description='Submit', layout=Layout(bottom='0px', grid…

In [12]:
footer_html = {
    'gap':'<br><br><br><br><br><br><hr>',

    'sjsu':'''Made by Dmytro Mishchenko as a project for the
    <a href='https://pytorch2021.devpost.com/'><u>PyTorch Hackathon
    </u> </a> with the help of a <a href='https://mlatsjsu.club'>
    <u>Machine Learning Club</u></a> at San Jose State University.''',

    'github':'''This project is open-source and is covered by the MIT License,
    for more details or if you want to contribure here is the
    <a href='https://github.com/mdmittriy/AgingClock'><u>
    GitHub repository</u></a>.''',

    'demo':'''You can take a look at the demo of this project <a
    href='https://www.youtube.com/watch?v=SCxRiXIG13Q'><u>here</u></a>. And
    notice that this work is heavily inspired by
    <a href='http://www.aging.ai'><u>aging.ai</u></a>'''}

widgets.HTML(value='<center>{}<br>{}<br>{}<br>{}</center>'.format(
    footer_html['gap'],footer_html['sjsu'],
    footer_html['github'],footer_html['demo']))

HTML(value="<center><br><br><br><br><br><br><hr><br>Made by Dmytro Mishchenko as a project for the\n    <a hre…